In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn/pppc')
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn')
import numpy as np
import torch
import matplotlib.pyplot as plt

import pppc
from pppc.configs import PtychoNNTrainingConfigDict
from pppc.ptychonn.trainer import PtychoNNTrainer, PtychoNNHyperparameterScanner
from pppc.ptychonn.dataset_handle import HDF5Dataset
from pppc.ptychonn.model import PtychoNNModel, PtychoNNPhaseOnlyModel

[2023-11-22 16:55:52,158] Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-11-22 16:55:52,160] NumExpr defaulting to 8 threads.


Create a baseline config dict. 

In [3]:
base_config_dict = PtychoNNTrainingConfigDict()
base_config_dict['batch_size_per_process'] = 64
base_config_dict['num_epochs'] = 1
base_config_dict['learning_rate_per_process'] = 1e-3
base_config_dict['optimizer'] = 'adam'
base_config_dict['model_save_dir'] = '../../trained_models/model_36SpiralDatasets'
base_config_dict['validation_ratio'] = 0.01
dataset = HDF5Dataset('data/data_test.h5', verbose=False)
base_config_dict['dataset'] = dataset
base_config_dict['model'] = None
base_config_dict['debug'] = False

In [4]:
param_scan_dict = {
    'model': [(PtychoNNPhaseOnlyModel, {}), (PtychoNNModel, {'n_levels': 3}), (PtychoNNModel, {'n_levels': 4}), ],
    'batch_size_per_process': [64, 32],
    'learning_rate_per_process': [1e-3, 1e-4]
}

In [5]:
scanner = PtychoNNHyperparameterScanner(param_scan_dict, base_config_dict)
scanner.build()

In [6]:
scanner.result_table

,model,batch_size_per_process,learning_rate_per_process,min_val_loss_total,min_val_loss_mag,min_val_loss_phase,epoch_min_val_loss
0,PtychoNNPhaseOnlyModel,64,0.0010,0.0,0.0,0.0,0.0
1,PtychoNNPhaseOnlyModel,64,0.0001,0.0,0.0,0.0,0.0
2,PtychoNNPhaseOnlyModel,32,0.0010,0.0,0.0,0.0,0.0
3,PtychoNNPhaseOnlyModel,32,0.0001,0.0,0.0,0.0,0.0
4,PtychoNNModel_nLevels_3,64,0.0010,0.0,0.0,0.0,0.0
5,PtychoNNModel_nLevels_3,64,0.0001,0.0,0.0,0.0,0.0
6,PtychoNNModel_nLevels_3,32,0.0010,0.0,0.0,0.0,0.0
7,PtychoNNModel_nLevels_3,32,0.0001,0.0,0.0,0.0,0.0
8,PtychoNNModel_nLevels_4,64,0.0010,0.0,0.0,0.0,0.0
9,PtychoNNModel_nLevels_4,64,0.0001,0.0,0.0,0.0,0.0


In [ ]:
scanner.run()
scanner.result_table

  0%|                                                                                                                                                                                                                                                    | 0/12 [00:00<?, ?it/s][2023-11-22 16:55:53,185] Using DataParallel with 2 devices.
[2023-11-22 16:56:16,717] Saving improved model after Val Loss improved from inf to 0.54966
  8%|███████████████████▋                                                                                                                                                                                                                        | 1/12 [00:23<04:21, 23.77s/it][2023-11-22 16:56:16,857] Using DataParallel with 2 devices.


Device 0 memory info:
    8080.375 MB total
    1468.0 MB reserved
    11.21533203125 MB allocated


[2023-11-22 16:56:38,546] Saving improved model after Val Loss improved from inf to 0.54921
 17%|███████████████████████████████████████▎                                                                                                                                                                                                    | 2/12 [00:45<03:46, 22.62s/it][2023-11-22 16:56:38,679] Using DataParallel with 2 devices.


Device 0 memory info:
    8080.375 MB total
    1492.0 MB reserved
    22.4814453125 MB allocated


[2023-11-22 16:57:02,749] Saving improved model after Val Loss improved from inf to 0.53711
 25%|███████████████████████████████████████████████████████████                                                                                                                                                                                 | 3/12 [01:09<03:29, 23.32s/it][2023-11-22 16:57:02,833] Using DataParallel with 2 devices.


Device 0 memory info:
    8080.375 MB total
    486.0 MB reserved
    33.69677734375 MB allocated


In [ ]:
#scanner.plot_all_training_history()
scanner.run_testing_for_all((0, 1))